In [226]:
%pip install langchain
%pip install langgraph
%pip install ag-ui-protocol


/Users/Guruprasad.BR/workspace/RD/fds-documentation-explorer/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/Guruprasad.BR/workspace/RD/fds-documentation-explorer/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/Guruprasad.BR/workspace/RD/fds-documentation-explorer/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [227]:
import pickle,os, traceback,json

all_events=[]
with open("all_events.pkl", "rb") as file:
    all_events=pickle.load(file)
# 
print(json.dumps(all_events[-2],default=str,indent=2))
print(len(all_events))



{
  "event": "on_chain_stream",
  "run_id": "dd8ecbed-c72d-4b77-916f-e495eb5efd76",
  "name": "fds_agent",
  "tags": [],
  "metadata": {
    "thread_id": "10f2f190-01f4-4d68-99d4-76d88e9e65ab",
    "user_id": "guru"
  },
  "data": {
    "chunk": {
      "end_conv": null
    }
  },
  "parent_ids": []
}
458


In [228]:

from langgraph.types import Command,Interrupt
from langchain_core import messages as langchain_messages
import copy

print(len(all_events))

def get_filtered_data(all_events):
    event_types={}
    chunk_types={}
    parsed_events={}
    unparsed_events={}
    filtered_events=[]
    for event in all_events:
        if "event" in event:
            _type=event["event"]
            event_types[_type]=event_types.get(_type,0)+1
            _contents=None
            # if _type in ["on_chain_start","on_chain_stream","on_chain_end","on_chat_model_start","on_chat_model_end"]:
            #     unparsed_events[_type]=unparsed_events.get(_type,0)+1
            #     continue
            if isinstance(event.get("data",{}).get("chunk",{}),langchain_messages.BaseMessage):
                ai_msg:langchain_messages.AIMessage=event.get("data",{}).get("chunk",{})
                _contents=ai_msg.content
                pass            
            elif event.get("data") and event["data"].get("output") and type(event["data"]["output"])==Command:          
                unparsed_events[_type]=unparsed_events.get(_type,0)+1
                continue
            elif (event.get("data") and event["data"].get("chunk") and type(event["data"]["chunk"])==Command):
                unparsed_events[_type]=unparsed_events.get(_type,0)+1
                continue
            elif event.get("data",{}).get("chunk",{}).get("__interrupt__",False):
                _interrupt:Interrupt= event["data"]["chunk"]["__interrupt__"]
                _contents={
                    "type": "interrupt",
                    "value":_interrupt[0].value,
                    "id": _interrupt[0].id,
                }        
                # event=copy.deepcopy(event)
                # event["data"]["chunk"]["__interrupt__"]={
                #     "type": "interrupt",
                #     "value":_interrupt[0].value,
                #     "id": _interrupt[0].id,
                # }        
            elif _type in ["on_tool_start","on_tool_end"]:
                pass
            elif not isinstance(event.get("data",{}).get("chunk",{}),langchain_messages.BaseMessage):
                if event.get("data",{}).get("chunk",{}).get("start_conv",True):
                    unparsed_events[_type]=unparsed_events.get(_type,0)+1
                    continue
                if event.get("data",{}).get("chunk",{}).get("llm",True):
                    unparsed_events[_type]=unparsed_events.get(_type,0)+1
                    continue
                if event.get("data",{}).get("chunk",{}).get("route",True):
                    unparsed_events[_type]=unparsed_events.get(_type,0)+1
                    continue        
                print("Unhandled", event)       
            else:
                _contents=event.get("data",{}).get("chunk",{}).get("content")

            filtered_events.append(event)
            parsed_events[_type]=parsed_events.get(_type,0)+1       
            if _type=="on_chat_model_stream" and _contents is not None and type(_contents) is list:            
                for _content in _contents:
                    if "type" in _content:
                        chunk_types[_type]=chunk_types.get(_type,set()) | {_content["type"]}
            else:
                chunk_types[_type]=chunk_types.get(_type,0)+1

    # print("event_types", event_types)
    # print("chunk_types", chunk_types)
    # print("parsed_events", parsed_events)
    # print("unparsed_events", unparsed_events)
    # print("filtered_events", len(filtered_events))
    # with open("filtered_data.json", "w") as file:
    #     file.write(json.dumps(filtered_events, default=str,indent=2))
    return filtered_events

get_filtered_data(all_events)

458


[{'event': 'on_chat_model_stream',
  'data': {'chunk': AIMessageChunk(content=[], additional_kwargs={}, response_metadata={}, id='run--626ac83e-eafc-4060-bb3f-db85fc113ec9')},
  'run_id': '626ac83e-eafc-4060-bb3f-db85fc113ec9',
  'name': 'ChatBedrockConverse',
  'tags': ['seq:step:1'],
  'metadata': {'thread_id': '10f2f190-01f4-4d68-99d4-76d88e9e65ab',
   'user_id': 'guru',
   'langgraph_step': 4,
   'langgraph_node': 'llm',
   'langgraph_triggers': ('branch:to:llm',),
   'langgraph_path': ('__pregel_pull', 'llm'),
   'langgraph_checkpoint_ns': 'llm:f930a039-9322-3393-5d92-82d9549bdcb4',
   'checkpoint_ns': 'llm:f930a039-9322-3393-5d92-82d9549bdcb4',
   'ls_provider': 'amazon_bedrock',
   'ls_model_name': 'us.anthropic.claude-sonnet-4-20250514-v1:0',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0,
   'ls_max_tokens': 20000},
  'parent_ids': ['dd8ecbed-c72d-4b77-916f-e495eb5efd76',
   '07c5fef1-6c9b-4d31-9778-4c57ae081245']},
 {'event': 'on_chat_model_stream',
  'data': {'chunk': 

In [229]:
from ag_ui.core import (
    EventType,
    CustomEvent,
    MessagesSnapshotEvent,
    RawEvent,
    RunAgentInput,
    RunErrorEvent,
    RunFinishedEvent,
    RunStartedEvent,
    StateDeltaEvent,
    StateSnapshotEvent,
    StepFinishedEvent,
    StepStartedEvent,
    TextMessageContentEvent,
    TextMessageEndEvent,
    TextMessageStartEvent,
    ToolCallArgsEvent,
    ToolCallEndEvent,
    ToolCallStartEvent,
    ThinkingTextMessageStartEvent,
    ThinkingTextMessageContentEvent,
    ThinkingTextMessageEndEvent,
    ThinkingStartEvent,
    ThinkingEndEvent,
)

In [230]:
from enum import Enum
class EventType(str, Enum):
    """
    The type of event.
    """
    TEXT_MESSAGE_START = "TEXT_MESSAGE_START"
    TEXT_MESSAGE_CONTENT = "TEXT_MESSAGE_CONTENT"
    TEXT_MESSAGE_END = "TEXT_MESSAGE_END"
    TEXT_MESSAGE_CHUNK = "TEXT_MESSAGE_CHUNK"
    THINKING_TEXT_MESSAGE_START = "THINKING_TEXT_MESSAGE_START"
    THINKING_TEXT_MESSAGE_CONTENT = "THINKING_TEXT_MESSAGE_CONTENT"
    THINKING_TEXT_MESSAGE_END = "THINKING_TEXT_MESSAGE_END"
    TOOL_CALL_START = "TOOL_CALL_START"
    TOOL_CALL_ARGS = "TOOL_CALL_ARGS"
    TOOL_CALL_END = "TOOL_CALL_END"
    TOOL_CALL_CHUNK = "TOOL_CALL_CHUNK"
    TOOL_CALL_RESULT = "TOOL_CALL_RESULT"
    THINKING_START = "THINKING_START"
    THINKING_END = "THINKING_END"
    STATE_SNAPSHOT = "STATE_SNAPSHOT"
    STATE_DELTA = "STATE_DELTA"
    MESSAGES_SNAPSHOT = "MESSAGES_SNAPSHOT"
    RAW = "RAW"
    CUSTOM = "CUSTOM"
    RUN_STARTED = "RUN_STARTED"
    RUN_FINISHED = "RUN_FINISHED"
    RUN_ERROR = "RUN_ERROR"
    STEP_STARTED = "STEP_STARTED"
    STEP_FINISHED = "STEP_FINISHED"

In [231]:
print(event_types)
print(chunk_types)
from datetime import datetime, timezone



{'on_chain_start': 22, 'on_chain_stream': 41, 'on_chain_end': 21, 'on_chat_model_start': 5, 'on_chat_model_stream': 80, 'on_chat_model_end': 5, 'on_tool_start': 5, 'on_tool_end': 5}
{'on_chat_model_stream': {'text', 'tool_use'}, 'on_tool_start': 5, 'on_tool_end': 5, 'on_chain_stream': 1}


In [232]:
import nest_asyncio
import asyncio
nest_asyncio.apply()
from ag_ui.encoder import EventEncoder
%xmode Verbose


def process_chunk(event,_type,_content,ref):
    # print(_content)
    last_type=ref.get("last_type",_type)
    ac_events=[]
    if _type!= last_type and last_type is not None and ref["wrap_status"]=="started":
        if last_type=="text":
            ac_events.append(TextMessageEndEvent(
                type=EventType.TEXT_MESSAGE_END,
                message_id=""
            ))
        elif last_type=="tool_use":
            ac_events.append(CustomEvent(
                type=EventType.CUSTOM,
                name=last_type,
                value={"text":"","type":EventType.TEXT_MESSAGE_END}
            ))

        ref["wrap_status"]="ended"
    if _type!= last_type or ref["wrap_status"] != "started":
        if _type=="text":
            ac_events.append(TextMessageStartEvent(
                type=EventType.TEXT_MESSAGE_START,
                role='assistant',
                message_id=""
            ))
        elif _type=="tool_use":
            ac_events.append(CustomEvent(
                type=EventType.CUSTOM,
                name=_type,
                value={"text":"","type":EventType.TEXT_MESSAGE_START},
            ))
        ref["wrap_status"]="started"    
    if not _content:
        ref["wrap_status"]="ended"
    if _content:
        if _type=="text":
            if _content.get("text"):
                ac_events.append(TextMessageContentEvent(
                    type=EventType.TEXT_MESSAGE_CONTENT,
                    message_id=_content.get("id", ""),
                    # raw_event=event,
                    delta= _content.get("text"),
                ))
        elif _type=="tool_use":
            if _content.get("id") and _content.get("name"):
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    # raw_event=event,
                    name=_type,
                    value= {"text":f"Proposed Tool Call: Name: {_content["name"]}, Id: {_content["id"]}","type":EventType.TEXT_MESSAGE_CONTENT}
                ))
            elif _content.get("input"):
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    # raw_event=event,
                    name=_type,
                    value= {"text":f"Arguments: {_content["input"]}","type":EventType.TEXT_MESSAGE_CONTENT}
                ))
        else:
            print("Unhandled chunk type:", _type, _content)
    return ac_events
def process_chunks(event,ref):
    chunks=[]
    if event["event"]=="on_chat_model_stream":
        base_message:langchain_messages.BaseMessage=event.get("data",{}).get("chunk",{})  
        _contents=base_message.content
        for i in range(len(_contents)):
            _content=_contents[i]
            if "type" in _content:
                chunks.extend(process_chunk(event, _content["type"], _content,ref))
                ref["last_type"] = _content["type"]
    else:
        chunks.extend(process_chunk(event, event["event"], None,ref))
    return chunks
    
def process_non_chunks(event,ref):
    ac_events=[]

    if event["event"]=="on_tool_start":
        ref["wrap_status"]="started"
        ref["last_type"]="tool_call"
        ac_events.append(ToolCallStartEvent(
            type=EventType.TOOL_CALL_START,
            tool_call_name=event["name"],
            tool_call_id="",
            raw_event=event
        ))
        ac_events.append(ToolCallArgsEvent(
            type=EventType.TOOL_CALL_ARGS,
            delta=json.dumps(event["data"]["input"], default=str),
            tool_call_id="",
            raw_event=event
        ))
    elif event["event"]=="on_tool_end":
        ref["wrap_status"]="ended"
        ref["last_type"]="tool_call"
        tool_message:langchain_messages.ToolMessage=event["data"]["output"]
        ac_events.append(ToolCallEndEvent(
            type=EventType.TOOL_CALL_END,
            tool_call_id=tool_message.tool_call_id,
            raw_event=event
        ))
    elif event.get("data",{}).get("chunk",{}).get("__interrupt__",False):
        _interrupt:Interrupt= event["data"]["chunk"]["__interrupt__"]
        ac_events.append(CustomEvent(
            type=EventType.CUSTOM,
            name="on_interrupt",
            value={
                "text": _interrupt[0].value,
                "id": _interrupt[0].id,
                "type": "on_interrupt"
            },
            raw_event=event
        ))
    else:
        print("Unhandled event:", event)
    return ac_events

async def handle_event(event,ref):
    chunks=[]
    chunks.extend(process_chunks(event,ref))
    if not isinstance(event.get("data",{}).get("chunk",{}),langchain_messages.BaseMessage): 
        chunks.extend(process_non_chunks(event,ref))
    if chunks:
        for chunk in chunks:
            yield chunk

async def handle_events(events):    
    ref={"wrap_status":None,"last_type":None}
    for event in events:
        async for transformed_event in handle_event(event,ref):
            yield transformed_event
    
    # if ref["wrap_status"]=="started":
    #     if ref["last_type"] =="text":
    #         yield TextMessageEndEvent(
    #             type=EventType.TEXT_MESSAGE_END,
    #             message_id=""
    #         )
    #     if ref["last_type"] =="tool_use":
    #         yield CustomEvent(
    #             type=EventType.CUSTOM,
    #             name=ref["last_type"],
    #             value={"text":"","type":EventType.TEXT_MESSAGE_END}
    #         )

    #     ref["wrap_status"]=None

# asyncio.run(handle_events())
try:
    filtered_events=get_filtered_data(all_events)
    encoder = EventEncoder(accept="text/event-stream")
    final_events=[encoder.encode(item) async for item in handle_events(filtered_events)]
    print("Final Events Count:", len(final_events))
    with open("final_events.json", "w") as file:
        file.write(json.dumps(final_events, default=str,indent=2))
except Exception as e:
    traceback.print_exc(limit=None)

Exception reporting mode: Verbose
Final Events Count: 362
